In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [2]:
seed = 23
protected = False
sam = False
import_cols = [
    'RESEARCH_AND_DEVELOPMENT',
    'DOMESTIC_SHELTER',
    'TYPE_OF_SET_ASIDE',
    'SOLICITATION_ID',
    'CONSTRUCTION_FIRM',
    'CO_BUS_SIZE_DETERMINATION',
    'CAGE_CODE',
    'VETERAN_OWNED_FLAG',
    'CORP_ENTITY_NOT_TAX_EXEMPT',
    'FUNDING_DEPARTMENT_ID',
    # 'FUNDING_AGENCY_NAME',
    'FUNDING_AGENCY_ID',
    # 'FUNDING_OFFICE_NAME',
    # 'FUNDING_OFFICE_ID',
    'SERVICE_PROVIDER',
    'PRODUCT_OR_SERVICE_TYPE',
    # 'MODIFICATION_NUMBER',
    'PIID',
    'FOUNDATION',
    # 'EVALUATED_PREFERENCE',
    'SRDVOB_FLAG',
    'CORP_ENTITY_TAX_EXEMPT',
    'MANUFACTURER_OF_GOODS',
    'VENDOR_ADDRESS_COUNTRY_NAME',
    'VENDOR_ADDRESS_ZIP_CODE',
    'SDB',
    'VETERINARY_HOSPITAL',
    'COMMUNITY_CORP_OWNED_FIRM',
    'DOT_CERTIFIED_DISADV_BUS',
    'PRINCIPAL_NAICS_CODE',
    'EDUCATIONAL_INSTITUTION_FLAG',
    'LIMITED_LIABILITY_CORPORATION',
    'EXTENT_COMPETED',
    'FEDERALLY_FUNDED_R_AND_D_CORP',
    'SOLE_PROPREITORSHIP',
    'WOMEN_OWNED_FLAG',
    'ARCHITECTURE_AND_ENGINEERING',
    'HISPANIC_SERVICING_INSTITUTION',
    'IDV_PIID',
    'PLACE_OF_MANUFACTURE',
    # 'IDV_EXTENT_COMPETED',
    'AWARD_FISCAL_YEAR',
    # 'IDV_SIGNED_DATE',
    'FIRM_8A_FLAG',
    'SMALL_AGRICULTURAL_COOPERATIVE',
    'PARTNERSHIP_OR_LLP',
    'DOLLARS_OBLIGATED',
    # 'IDV_NUMBER_OF_OFFERS',
    'FOR_PROFIT_ORGANIZATION',
    # 'AWARD_OR_IDV',
    'FIRM8A_JOINT_VENTURE',
    # 'IDV_CONTRACTING_AGENCY_ID',
    'NUMBER_OF_OFFERS_RECEIVED'
]
if protected:
    import_cols = import_cols + ['ANNUAL_REVENUE', 'NUMBER_OF_EMPLOYEES']


import_years = [2019, 2020, 2021, 2022, 2023]
top_n_agencies = 0
min_class_size = 20

model_cols = [
    'RESPONSE',
    # 'RESEARCH_AND_DEVELOPMENT',
    # 'DOMESTIC_SHELTER',
    # 'TYPE_OF_SET_ASIDE',
    # 'CONSTRUCTION_FIRM',
    # 'VETERAN_OWNED_FLAG',
    # 'CORP_ENTITY_NOT_TAX_EXEMPT',
    # 'SERVICE_PROVIDER',
    # 'PRODUCT_OR_SERVICE_TYPE',
    # 'MODIFICATION_NUMBER',
    # 'PIID',
    # 'FOUNDATION',
    # 'EVALUATED_PREFERENCE',
    # 'SRDVOB_FLAG',
    # 'CORP_ENTITY_TAX_EXEMPT',
    # 'MANUFACTURER_OF_GOODS',
    # 'VENDOR_ADDRESS_ZIP_CODE',
    #'VENDOR_STATE', #FEATURE ENGINEERED BELOW
    # 'SDB',
    # 'VETERINARY_HOSPITAL',
    # 'COMMUNITY_CORP_OWNED_FIRM',
    # 'DOT_CERTIFIED_DISADV_BUS',
    'PRINCIPAL_NAICS_CODE',
    # 'EDUCATIONAL_INSTITUTION_FLAG',
    # 'LIMITED_LIABILITY_CORPORATION',
    # 'FEDERALLY_FUNDED_R_AND_D_CORP',
    # 'SOLE_PROPREITORSHIP',
    # 'WOMEN_OWNED_FLAG',
    # 'ARCHITECTURE_AND_ENGINEERING',
    # 'HISPANIC_SERVICING_INSTITUTION',
    # 'IDV_PIID',
    # 'PLACE_OF_MANUFACTURE_CLASS',
    # 'IDV_EXTENT_COMPETED',
    # 'AWARD_FISCAL_YEAR',
    # 'IDV_SIGNED_DATE',
    # 'FIRM_8A_FLAG',
    # 'SMALL_AGRICULTURAL_COOPERATIVE',
    # 'PARTNERSHIP_OR_LLP',
    # 'IDV_NUMBER_OF_OFFERS',
    # 'FOR_PROFIT_ORGANIZATION',
    # 'AWARD_OR_IDV',
    # 'FIRM8A_JOINT_VENTURE',
    # 'IDV_CONTRACTING_AGENCY_ID',
    # 'CONTRACTS_PER_YEAR'
]
if protected:
    model_cols = model_cols + ['ANNUAL_REVENUE', 'NUMBER_OF_EMPLOYEES']

min_importance = 0.001 #at 0.015 we finally get zip, 
col_tree_depth = 10
hard = False

n_trees = 250
max_depth = 25
top_n_classes = 20

## Import Data

In [3]:
def import_dataset(import_cols, years, sam=True):
    """
    Imports, cleans, and joins our data with specified columns and years
    Inputs: 
        import_cols (list [str] of column names)
        years (list [int] of years to import)
        sam (bool of whether to merge with sam dataset)
    Output:
        Cleaned, filtered, and joined dataframe
    """
    if sam:
        SAM = pd.read_csv('SAM.CSV') #imports SAM df
    
    year_dfs = []
    for year in years:
        if protected: #whether we're using protected columns or not
            temp_df = pd.read_parquet('fy' + str(year) + '.parquet', columns=import_cols)
        else:
            temp_df = pd.read_parquet(str(year) + '.parquet', columns=import_cols) #import year's data
        
        temp_df = temp_df[temp_df['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"] #filter for small business
        temp_df = temp_df[temp_df['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"] #filter for US
        temp_df = temp_df[temp_df['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])] #filter for competition
        
        # temp_df['DOLLARS_OBLIGATED'] = pd.to_numeric(temp_df['DOLLARS_OBLIGATED'], errors='coerce') #make numeric
        
        if sam:
            temp_m = pd.merge(temp_df, SAM, on="CAGE_CODE", how="inner") #merge with SAM
        else:
            temp_m = temp_df
        
        # idx = temp_m.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax() #find initial contract win
        # temp_m = temp_m.loc[idx] #filter to initial contract win
        
        temp_m = temp_m[temp_m['DOLLARS_OBLIGATED'] > 0] #filter DOLLARS_OBLIGATED
        
        print(f'{year} shape: {temp_m.shape}')
        year_dfs.append(temp_m) #append year dataset to list of year datasets
    
    merged_df = pd.concat(year_dfs, ignore_index=True) #merge all years
    
    for df in year_dfs:
        del df
    del year_dfs #delete the individual dfs from memory
    
    merged_df['contract_id'] = merged_df['PIID'] + merged_df['IDV_PIID']
    # idx = merged_df.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax() #find initial contracts
    idx = merged_df.groupby(['contract_id', 'CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax() #find initial contracts a different way lmao
    filtered_merged_df = merged_df.loc[idx] #filter to initial contract
    
    print(f'total shape: {filtered_merged_df.shape}')
    
    #place of manufacture conversion
    def convert_place_of_manufacture(value):
        if value == 'D':
            return 'YES' #manufactured in US
        elif value == 'C':
            return 'NO' #not manufactured in US
        elif value in ['N/A', 'A', 'G', 'E', 'H', 'L', 'J', 'F', 'K', 'B', 'I']:
            return 'NONE'
        else:
            return 'NONE' #N/A (provides a service or doesn't qualify fully)
    
    if protected:
        filtered_merged_df['ANNUAL_REVENUE'] = filtered_merged_df['ANNUAL_REVENUE'].astype(float)
    # filtered_merged_df['IDV_PIID'] = filtered_merged_df['IDV_PIID'].str.strip() #clean IDV PIID
    # filtered_merged_df['PIID'] = filtered_merged_df['PIID'].str.strip() #clean PIID
    filtered_merged_df['PLACE_OF_MANUFACTURE_CLASS'] = filtered_merged_df['PLACE_OF_MANUFACTURE'].apply(convert_place_of_manufacture) #clean PLACE_OF_MANUFACTURE
    filtered_merged_df['VENDOR_ADDRESS_ZIP_CODE'] = filtered_merged_df['VENDOR_ADDRESS_ZIP_CODE'].astype(str).str[:5] #clean ZIP to 5-digit
    filtered_merged_df = filtered_merged_df[filtered_merged_df['VENDOR_ADDRESS_ZIP_CODE'].str.len()==5]
    filtered_merged_df['TYPE_OF_SET_ASIDE'] = filtered_merged_df['TYPE_OF_SET_ASIDE'].fillna('NONE') #assume NA = NONE
    
    filtered_merged_df = filtered_merged_df.dropna(subset=filtered_merged_df.columns.difference(['PLACE_OF_MANUFACTURE'])) #remove rows with NAs here
    
    print(f'total filtered shape: {filtered_merged_df.shape}')
    
    return filtered_merged_df

In [4]:
df0 = import_dataset(import_cols, import_years, sam=False)

2019 shape: (1772611, 43)
2020 shape: (1659942, 43)
2021 shape: (1916684, 43)
2022 shape: (1979453, 43)
2023 shape: (1866515, 43)
total shape: (8616491, 44)
total filtered shape: (266059, 45)


## Create Response Variable

In [5]:
def response_var(df, top_n, min_class_size=1):
    """
    Creates a new response variable by splitting the top_n most common agencies into their respective offices,
    then filter the dataframe so only classes with at least min_class_size rows remain
    Inputs:
        df (dataframe output from import_dataset())
        top_n (integer of top n agencies [0, max])
        min_class_size (integer of minimum # of observations a class can have to be a valid response)
    Outputs:
        dataframe with additional RESPONSE column, filtered
    """
    if top_n > 0:
        n_agencies = df['FUNDING_AGENCY_ID'].value_counts().nlargest(top_n).index #get top n most common agencies
        df['RESPONSE'] = np.where(df['FUNDING_AGENCY_ID'].isin(n_agencies), df['FUNDING_OFFICE_ID'], df['FUNDING_AGENCY_ID']) #make RESPONSE, splitting most common agencies into offices
    else:
        df['RESPONSE'] = df['FUNDING_AGENCY_ID']
    
    class_sizes = df['RESPONSE'].value_counts()
    classes_to_keep = class_sizes[class_sizes >= min_class_size].index
    df = df[df['RESPONSE'].isin(classes_to_keep)]
    
    print(f"{df['RESPONSE'].nunique()} unique response classes")
    
    return df

In [6]:
df = response_var(df0, top_n_agencies, min_class_size)

163 unique response classes


## Perform any Feature Engineering Here

In [7]:
df.loc[:,'PRINCIPAL_NAICS_CODE'] = df['PRINCIPAL_NAICS_CODE'].str[:4]

In [8]:
# contracts_per_year = df.groupby(['CAGE_CODE', 'AWARD_FISCAL_YEAR']).size().to_frame().reset_index()
# contracts_per_year.columns = list(contracts_per_year.columns[:2]) + ['CONTRACTS_PER_YEAR']
# df = df.merge(contracts_per_year, on=['CAGE_CODE', 'AWARD_FISCAL_YEAR'])

In [9]:
# zip_df = pd.read_csv('zip_code_database.csv', converters={'zip': str})
# zcdb = pd.Series(zip_df['state'].values,index=zip_df['zip']).to_dict()
# df.loc[:,'VENDOR_STATE'] = df["VENDOR_ADDRESS_ZIP_CODE"].map(zcdb, na_action='ignore')
# df = df[~df['VENDOR_STATE'].isna()]

In [10]:
# df.loc[:,'VENDOR_ADDRESS_ZIP_CODE'] = df['VENDOR_ADDRESS_ZIP_CODE'].str[:1]

## Model Preprocessing

In [11]:
def preprocess_data(df, model_cols, train_test=True, train_size=0.8, scale_quant=True):
    """
    Preprocesses the dataframe appropriately after creating the response variable and after feature engineering
    Input:
        df (dataframe output from response_var())
        model_cols (list [str] of cols to use to model)
        train_test (bool of whether to split the data into train/test groups)
        train_size (float [0, 1] of proportion of training data to whole data)
        scale_quant (bool of whether to scale quantitative variables)
        
    Output:
        X (df of X values to be used in modeling, dummy encoded & scaled)
        y (series of y values for modeling, label encoded)
    """
    Xy = df[model_cols] #select only the modeling columns
    
    y = Xy['RESPONSE'] #initialize y
    X = Xy.drop('RESPONSE', axis=1) #initialize X
    
    enc = LabelEncoder()
    y = enc.fit_transform(y) #transform y into labeled column
    global class_ids 
    class_ids = enc.classes_
    
    categoricals = X.select_dtypes(include=['object', 'category']).columns.tolist()
    X_cat = pd.get_dummies(X[categoricals]) #one-hot encode categorical columns
    
    quantitatives = X.columns.difference(categoricals)
    if scale_quant and len(quantitatives) > 0:
        global scaler
        scaler = StandardScaler()
        X_quant = scaler.fit_transform(X[quantitatives]) #scale quantitative columns
        X_quant = pd.DataFrame(X_quant, columns=quantitatives)
        X_cat = X_cat.reset_index()
    else:
        X_quant = X[quantitatives]
    
    X = pd.concat([X_cat, X_quant], axis=1) #combine quant & cat subsets back into one df
    
    if train_test:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=seed)
        return X_train, X_test, y_train, y_test
    else:
        return X, y

In [12]:
X_train, X_test, y_train, y_test = preprocess_data(df, model_cols, train_test=True, train_size=0.8, scale_quant=True)

In [13]:
# pickle.dump(scaler, open('Dashboard_Scaler.pkl', 'wb'))

## Column Selection

In [14]:
final_cols = [
    'PRINCIPAL_NAICS_CODE_1129',
 'PRINCIPAL_NAICS_CODE_1132',
 'PRINCIPAL_NAICS_CODE_1151',
 'PRINCIPAL_NAICS_CODE_1152',
 'PRINCIPAL_NAICS_CODE_1153',
 'PRINCIPAL_NAICS_CODE_2123',
 'PRINCIPAL_NAICS_CODE_2131',
 'PRINCIPAL_NAICS_CODE_2211',
 'PRINCIPAL_NAICS_CODE_2362',
 'PRINCIPAL_NAICS_CODE_2371',
 'PRINCIPAL_NAICS_CODE_2373',
 'PRINCIPAL_NAICS_CODE_2379',
 'PRINCIPAL_NAICS_CODE_2381',
 'PRINCIPAL_NAICS_CODE_2382',
 'PRINCIPAL_NAICS_CODE_2383',
 'PRINCIPAL_NAICS_CODE_2389',
 'PRINCIPAL_NAICS_CODE_3111',
 'PRINCIPAL_NAICS_CODE_3112',
 'PRINCIPAL_NAICS_CODE_3114',
 'PRINCIPAL_NAICS_CODE_3115',
 'PRINCIPAL_NAICS_CODE_3116',
 'PRINCIPAL_NAICS_CODE_3117',
 'PRINCIPAL_NAICS_CODE_3119',
 'PRINCIPAL_NAICS_CODE_3121',
 'PRINCIPAL_NAICS_CODE_3132',
 'PRINCIPAL_NAICS_CODE_3133',
 'PRINCIPAL_NAICS_CODE_3141',
 'PRINCIPAL_NAICS_CODE_3149',
 'PRINCIPAL_NAICS_CODE_3151',
 'PRINCIPAL_NAICS_CODE_3152',
 'PRINCIPAL_NAICS_CODE_3159',
 'PRINCIPAL_NAICS_CODE_3162',
 'PRINCIPAL_NAICS_CODE_3169',
 'PRINCIPAL_NAICS_CODE_3219',
 'PRINCIPAL_NAICS_CODE_3221',
 'PRINCIPAL_NAICS_CODE_3222',
 'PRINCIPAL_NAICS_CODE_3231',
 'PRINCIPAL_NAICS_CODE_3241',
 'PRINCIPAL_NAICS_CODE_3251',
 'PRINCIPAL_NAICS_CODE_3252',
 'PRINCIPAL_NAICS_CODE_3253',
 'PRINCIPAL_NAICS_CODE_3254',
 'PRINCIPAL_NAICS_CODE_3255',
 'PRINCIPAL_NAICS_CODE_3256',
 'PRINCIPAL_NAICS_CODE_3259',
 'PRINCIPAL_NAICS_CODE_3261',
 'PRINCIPAL_NAICS_CODE_3262',
 'PRINCIPAL_NAICS_CODE_3271',
 'PRINCIPAL_NAICS_CODE_3273',
 'PRINCIPAL_NAICS_CODE_3279',
 'PRINCIPAL_NAICS_CODE_3313',
 'PRINCIPAL_NAICS_CODE_3314',
 'PRINCIPAL_NAICS_CODE_3322',
 'PRINCIPAL_NAICS_CODE_3323',
 'PRINCIPAL_NAICS_CODE_3324',
 'PRINCIPAL_NAICS_CODE_3325',
 'PRINCIPAL_NAICS_CODE_3326',
 'PRINCIPAL_NAICS_CODE_3327',
 'PRINCIPAL_NAICS_CODE_3328',
 'PRINCIPAL_NAICS_CODE_3329',
 'PRINCIPAL_NAICS_CODE_3331',
 'PRINCIPAL_NAICS_CODE_3332',
 'PRINCIPAL_NAICS_CODE_3333',
 'PRINCIPAL_NAICS_CODE_3334',
 'PRINCIPAL_NAICS_CODE_3335',
 'PRINCIPAL_NAICS_CODE_3336',
 'PRINCIPAL_NAICS_CODE_3339',
 'PRINCIPAL_NAICS_CODE_3341',
 'PRINCIPAL_NAICS_CODE_3342',
 'PRINCIPAL_NAICS_CODE_3343',
 'PRINCIPAL_NAICS_CODE_3344',
 'PRINCIPAL_NAICS_CODE_3345',
 'PRINCIPAL_NAICS_CODE_3346',
 'PRINCIPAL_NAICS_CODE_3351',
 'PRINCIPAL_NAICS_CODE_3352',
 'PRINCIPAL_NAICS_CODE_3353',
 'PRINCIPAL_NAICS_CODE_3359',
 'PRINCIPAL_NAICS_CODE_3361',
 'PRINCIPAL_NAICS_CODE_3362',
 'PRINCIPAL_NAICS_CODE_3363',
 'PRINCIPAL_NAICS_CODE_3364',
 'PRINCIPAL_NAICS_CODE_3365',
 'PRINCIPAL_NAICS_CODE_3366',
 'PRINCIPAL_NAICS_CODE_3369',
 'PRINCIPAL_NAICS_CODE_3371',
 'PRINCIPAL_NAICS_CODE_3372',
 'PRINCIPAL_NAICS_CODE_3379',
 'PRINCIPAL_NAICS_CODE_3391',
 'PRINCIPAL_NAICS_CODE_3399',
 'PRINCIPAL_NAICS_CODE_4221',
 'PRINCIPAL_NAICS_CODE_4231',
 'PRINCIPAL_NAICS_CODE_4232',
 'PRINCIPAL_NAICS_CODE_4233',
 'PRINCIPAL_NAICS_CODE_4234',
 'PRINCIPAL_NAICS_CODE_4236',
 'PRINCIPAL_NAICS_CODE_4237',
 'PRINCIPAL_NAICS_CODE_4238',
 'PRINCIPAL_NAICS_CODE_4239',
 'PRINCIPAL_NAICS_CODE_4241',
 'PRINCIPAL_NAICS_CODE_4244',
 'PRINCIPAL_NAICS_CODE_4247',
 'PRINCIPAL_NAICS_CODE_4249',
 'PRINCIPAL_NAICS_CODE_4251',
 'PRINCIPAL_NAICS_CODE_4412',
 'PRINCIPAL_NAICS_CODE_4413',
 'PRINCIPAL_NAICS_CODE_4431',
 'PRINCIPAL_NAICS_CODE_4441',
 'PRINCIPAL_NAICS_CODE_4442',
 'PRINCIPAL_NAICS_CODE_4481',
 'PRINCIPAL_NAICS_CODE_4511',
 'PRINCIPAL_NAICS_CODE_4529',
 'PRINCIPAL_NAICS_CODE_4532',
 'PRINCIPAL_NAICS_CODE_4539',
 'PRINCIPAL_NAICS_CODE_4541',
 'PRINCIPAL_NAICS_CODE_4811',
 'PRINCIPAL_NAICS_CODE_4812',
 'PRINCIPAL_NAICS_CODE_4831',
 'PRINCIPAL_NAICS_CODE_4841',
 'PRINCIPAL_NAICS_CODE_4842',
 'PRINCIPAL_NAICS_CODE_4853',
 'PRINCIPAL_NAICS_CODE_4854',
 'PRINCIPAL_NAICS_CODE_4859',
 'PRINCIPAL_NAICS_CODE_4881',
 'PRINCIPAL_NAICS_CODE_4883',
 'PRINCIPAL_NAICS_CODE_4889',
 'PRINCIPAL_NAICS_CODE_4921',
 'PRINCIPAL_NAICS_CODE_4922',
 'PRINCIPAL_NAICS_CODE_4931',
 'PRINCIPAL_NAICS_CODE_5111',
 'PRINCIPAL_NAICS_CODE_5112',
 'PRINCIPAL_NAICS_CODE_5121',
 'PRINCIPAL_NAICS_CODE_5131',
 'PRINCIPAL_NAICS_CODE_5132',
 'PRINCIPAL_NAICS_CODE_5151',
 'PRINCIPAL_NAICS_CODE_5171',
 'PRINCIPAL_NAICS_CODE_5172',
 'PRINCIPAL_NAICS_CODE_5173',
 'PRINCIPAL_NAICS_CODE_5174',
 'PRINCIPAL_NAICS_CODE_5179',
 'PRINCIPAL_NAICS_CODE_5181',
 'PRINCIPAL_NAICS_CODE_5182',
 'PRINCIPAL_NAICS_CODE_5191',
 'PRINCIPAL_NAICS_CODE_5221',
 'PRINCIPAL_NAICS_CODE_5223',
 'PRINCIPAL_NAICS_CODE_5231',
 'PRINCIPAL_NAICS_CODE_5239',
 'PRINCIPAL_NAICS_CODE_5242',
 'PRINCIPAL_NAICS_CODE_5259',
 'PRINCIPAL_NAICS_CODE_5311',
 'PRINCIPAL_NAICS_CODE_5312',
 'PRINCIPAL_NAICS_CODE_5313',
 'PRINCIPAL_NAICS_CODE_5321',
 'PRINCIPAL_NAICS_CODE_5322',
 'PRINCIPAL_NAICS_CODE_5323',
 'PRINCIPAL_NAICS_CODE_5324',
 'PRINCIPAL_NAICS_CODE_5411',
 'PRINCIPAL_NAICS_CODE_5412',
 'PRINCIPAL_NAICS_CODE_5413',
 'PRINCIPAL_NAICS_CODE_5414',
 'PRINCIPAL_NAICS_CODE_5415',
 'PRINCIPAL_NAICS_CODE_5416',
 'PRINCIPAL_NAICS_CODE_5417',
 'PRINCIPAL_NAICS_CODE_5418',
 'PRINCIPAL_NAICS_CODE_5419',
 'PRINCIPAL_NAICS_CODE_5611',
 'PRINCIPAL_NAICS_CODE_5612',
 'PRINCIPAL_NAICS_CODE_5613',
 'PRINCIPAL_NAICS_CODE_5614',
 'PRINCIPAL_NAICS_CODE_5615',
 'PRINCIPAL_NAICS_CODE_5616',
 'PRINCIPAL_NAICS_CODE_5617',
 'PRINCIPAL_NAICS_CODE_5619',
 'PRINCIPAL_NAICS_CODE_5621',
 'PRINCIPAL_NAICS_CODE_5622',
 'PRINCIPAL_NAICS_CODE_5629',
 'PRINCIPAL_NAICS_CODE_6113',
 'PRINCIPAL_NAICS_CODE_6114',
 'PRINCIPAL_NAICS_CODE_6115',
 'PRINCIPAL_NAICS_CODE_6116',
 'PRINCIPAL_NAICS_CODE_6117',
 'PRINCIPAL_NAICS_CODE_6211',
 'PRINCIPAL_NAICS_CODE_6212',
 'PRINCIPAL_NAICS_CODE_6213',
 'PRINCIPAL_NAICS_CODE_6215',
 'PRINCIPAL_NAICS_CODE_6219',
 'PRINCIPAL_NAICS_CODE_6221',
 'PRINCIPAL_NAICS_CODE_6231',
 'PRINCIPAL_NAICS_CODE_6241',
 'PRINCIPAL_NAICS_CODE_6242',
 'PRINCIPAL_NAICS_CODE_6243',
 'PRINCIPAL_NAICS_CODE_7111',
 'PRINCIPAL_NAICS_CODE_7121',
 'PRINCIPAL_NAICS_CODE_7139',
 'PRINCIPAL_NAICS_CODE_7211',
 'PRINCIPAL_NAICS_CODE_7212',
 'PRINCIPAL_NAICS_CODE_8111',
 'PRINCIPAL_NAICS_CODE_8112',
 'PRINCIPAL_NAICS_CODE_8113',
 'PRINCIPAL_NAICS_CODE_8114',
 'PRINCIPAL_NAICS_CODE_8129',
]

In [15]:
# pickle.dump(final_cols, open('Dashboard_Columns.pkl', 'wb'))

## Train Random Forest & Assess

In [16]:
def train_models(X_train, X_test, y_train, y_test, columns, top_n_classes, n_trees=1000, max_depth=2):
    X_train = X_train[columns]
    X_test = X_test[columns]
    
    # Initialize the Random Forest model
    random_forest = RandomForestClassifier(n_estimators=n_trees, max_depth=max_depth, random_state=seed,
                                           criterion="log_loss", class_weight="balanced_subsample", n_jobs=-1)
    # boost_model = GradientBoostingClassifier(n_estimators=40, learning_rate=0.05, max_depth=4, random_state=seed)

    # Train the Random Forest model on the training data
    random_forest.fit(X_train, y_train)
    # boost_model.fit(X_train, y_train)
    
    return None, random_forest

In [17]:
def assess_models(model):
    top_ns = []
    accuracies = []
    probabilities = model.predict_proba(X_test[final_cols])
    for i in [1, 5, 10, 15, 20]:
        # Get the top n predicted classes for each sample
        top_n_indices = np.argsort(probabilities, axis=1)[:, -i:]

        # Check if the true label is in the top n predicted classes for each sample
        predicted_labels = model.classes_[top_n_indices]
        accurate_predictions = np.any(predicted_labels == y_test[:, np.newaxis], axis=1)

        # Calculate accuracy based on whether the true label is in the top n predicted classes
        accuracy = np.mean(accurate_predictions)
        accuracies += [accuracy]
        top_ns += [i]
    
    return pd.DataFrame({'Top-n': top_ns, 'Accuracy': accuracies})

## NAICS Model

In [187]:
%%time
boosted_model, forest_model = train_models(X_train, X_test, y_train, y_test,
                                                columns=final_cols, top_n_classes=top_n_classes,
                                                n_trees=250, max_depth=25)

CPU times: user 1min 19s, sys: 1.22 s, total: 1min 20s
Wall time: 12.1 s


In [188]:
assess_df = assess_models(forest_model)
assess_df

,Top-n,Accuracy
0,1,0.496569
1,5,0.692946
2,10,0.737454
3,15,0.788467
4,20,0.822098


## Top Agency Model

In [96]:
def top_agency_model(n, repeats):
    topn_agencies = list(pd.Series(y_train).value_counts().head(n).index)
    accs = []
    for i in range(repeats):
        accs.append(pd.Series(y_test).apply(lambda x: x in topn_agencies).sum() / len(y_test))
    return np.mean(accs)

In [97]:
top_20 = top_agency_model(20, 1000)
top_15 = top_agency_model(15, 1000)
top_10 = top_agency_model(10, 1000)
top_5 = top_agency_model(5, 1000)
top_1= top_agency_model(1, 1000)

pd.DataFrame({'Top-n': [1, 5, 10, 15, 20],
             'Accuracy': [top_1, top_5, top_10, top_15, top_20]})

,Top-n,Accuracy
0,1,0.537724
1,5,0.797832
2,10,0.851128
3,15,0.879578
4,20,0.899393


## Empirical Distribution Random Model

In [84]:
def random_model(n):
    y_numpy = pd.Series(y_train).to_frame()\
    .merge(pd.Series(y_train).value_counts(normalize=True).to_frame().reset_index(),
           left_on=0, right_on='index')[['index', 'proportion']].drop_duplicates().to_numpy()
    y_probs = y_numpy[:,1]
    y_vals = y_numpy[:, 0]
    return np.random.choice(y_vals, n, p=y_probs, replace=False).astype(int)

In [19]:
y_numpy = pd.Series(y_train).to_frame()\
    .merge(pd.Series(y_train).value_counts(normalize=True).to_frame().reset_index(),
           left_on=0, right_on='index')[['index', 'proportion']].drop_duplicates().to_numpy()

In [78]:
def assess_random(n, repeats):
    accs = []
    for i in range(repeats):
        top_n_random = random_model(n)
        accs.append(pd.Series(y_test).apply(lambda x: x in top_n_random).sum() / len(y_test))
    
    return np.mean(accs)

In [81]:
random_20 = assess_random(20, 1000)
random_15 = assess_random(15, 1000)
random_10 = assess_random(10, 1000)
random_5 = assess_random(5, 1000)
random_1 = assess_random(1, 1000)

pd.DataFrame({'Top-n': [1, 5, 10, 15, 20],
             'Accuracy': [random_1, random_5, random_10, random_15, random_20]})

,Top-n,Accuracy
0,1,0.330721
1,5,0.709589
2,10,0.799872
3,15,0.833551
4,20,0.856998


## Fully Random Model

In [86]:
def random_model_full(n):
    y_vals = np.unique(y_train)
    return np.random.choice(y_vals, n, replace=False).astype(int)

In [87]:
def assess_random_full(n, repeats):
    accs = []
    for i in range(repeats):
        top_n_random = random_model_full(n)
        accs.append(pd.Series(y_test).apply(lambda x: x in top_n_random).sum() / len(y_test))
    
    return np.mean(accs)

In [88]:
random_full_20 = assess_random_full(20, 1000)
random_full_15 = assess_random_full(15, 1000)
random_full_10 = assess_random_full(10, 1000)
random_full_5 = assess_random_full(5, 1000)
random_full_1 = assess_random_full(1, 1000)

pd.DataFrame({'Top-n': [1, 5, 10, 15, 20],
             'Accuracy': [random_full_1, random_full_5, random_full_10, random_full_15, random_full_20]})

,Top-n,Accuracy
0,1,0.008811
1,5,0.025091
2,10,0.061811
3,15,0.082204
4,20,0.111644
